In [1]:
import numpy as np

In [2]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [4]:
# Initialize w and b with zeros

def initialize_with_zeros(dim):
    return np.zeros((dim, 1)), 0

In [6]:
# Propagate

def propagate(w, b, X, Y):
    m = X.shape[1]
    # Forward propagation (from X to cost)
    A = sigmoid(np.dot(w.T, X) + b)
    cost = -(Y * np.log(A) + (1 - Y) * np.log(1 - A)) / m
    
    # Backward propagation (to find gradients)
    dz = A - Y
    dw = np.dot(X, dz.T) / m
    db = np.sum(dz) / m
    
    grads = {"dw": dw, 
             "db": db}
    
    return grads, cost

In [8]:
# Optimize

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost=False):
    costs = []
    for i in range(num_iterations):
        # Get cost and gradients
        grads, cost = propagate(w, b, X, Y)
        
        # Retrive derivatives from gradients
        dw = grads['dw']
        db = grads['db']
        
        # Record costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [17]:
# Predict

def predict(w, b, X):
    w = w.reshape(X.shape[0], 1)
    A = sigmoid(np.dot(w.T, X) + b)
    return (A >= .5) + 0
    